<a href="https://colab.research.google.com/github/christoforou/CUS1166_Project_Template/blob/master/probabilistic_models/Pyro_Tutorials_Practicing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Practicing Pyro. 
The following tutorial in intented for getting familiary with the Pyro library and probabilitistic models. 

**References** 

- [Introduction to Inference with Pyro](https://pyro.ai/examples/intro_part_ii.html)
- [Setting up Pyro on Colab](https://medium.com/paper-club/how-to-set-up-google-colab-colaboratory-for-building-pyro-models-8e51129e772a)
- [Quickstart on Pytorch in Colab](https://medium.com/dair-ai/pytorch-1-2-quickstart-with-google-colab-6690a30c38d) 


### Installing PyTorch and Pyro

In [2]:
# 
# Pytoch Tutorial with Pyro

# Colab package installations
from os import path
#from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
#platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

#!pip3 install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.0-{platform}-linux_x86_64.whl

!pip3 install torch==1.2.0+cu92 torchvision==0.4.0+cu92 -f https://download.pytorch.org/whl/torch_stable.html
!pip3 install torchvision
!pip3 install pyro-ppl

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 663.1MB 28kB/s 
     |████████████████████████████████| 8.8MB 30.4MB/s 
ERROR: torchtext 0.9.0 has requirement torch==1.8.0, but you'll have torch 1.2.0+cu92 which is incompatible.
  Found existing installation: torch 1.8.0+cu101
    Uninstalling torch-1.8.0+cu101:
      Successfully uninstalled torch-1.8.0+cu101
  Found existing installation: torchvision 0.9.0+cu101
    Uninstalling torchvision-0.9.0+cu101:
      Successfully uninstalled torchvision-0.9.0+cu101
     |████████████████████████████████| 634kB 8.7MB/s 
     |████████████████████████████████| 735.5MB 24kB/s 
ERROR: torchvision 0.4.0+cu92 has requirement torch==1.2.0, but you'll have torch 1.8.0 which is incompatible.
  Found existing installation: torch 1.2.0+cu92
    Uninstalling torch-1.2.0+cu92:
      Successfully uninstalled torch-1.2.0+cu92


In [4]:
import torch
import pyro

pyro.set_rng_seed(101)

### 1. Introduction to Probabilistic Models 
The basic unit of probabilistic programs is the stochastic function. This is an arbitrary Python callable that combines two ingredients:
  - deterministic Python code; and
  - primitive stochastic functions that call a random number generator

Concretely, a stochastic function can be any Python object with a __call__() method, like a function, a method, or a PyTorch nn.Module.

Primitive stochastic functions, or distributions, are an important class of stochastic functions for which we can explicitly compute the probability of the outputs given the inputs.
```python 
loc = 0.   # mean zero
scale = 1. # unit variance
normal = torch.distributions.Normal(loc, scale) # create a normal distribution object
x = normal.rsample() # draw a sample from N(0,1)
print("sample", x)
print("log prob", normal.log_prob(x)) # score the sample from N(0,1)
```

In [6]:
# Examle code to generate sample from a Normal distribution   
loc = 0.   # mean zero
scale = 1. # unit variance
normal = torch.distributions.Normal(loc, scale) # create a normal distribution object
x = normal.rsample() # draw a sample from N(0,1)
print("sample", x)
print("log prob", normal.log_prob(x)) # score the sample from N(0,1)

sample tensor(-0.8152)
log prob tensor(-1.2512)


#### 1.1 Simple weather model - defining a joint probability 
All probabilistic programs are built up by composing primitive stochastic functions and deterministic computation.

Let’s suppose we have a bunch of data with daily mean temperatures and cloud cover. We want to reason about how temperature interacts with whether it was sunny or cloudy. A simple stochastic function that describes how that data might have been generated is given by 
```python 
def weather():
    cloudy = torch.distributions.Bernoulli(0.3).sample()    # draw a sample from a Bernoulli 0 or 1 
    cloudy = 'cloudy' if cloudy.item() == 1.0 else 'sunny'  # Transform cloudt into 'cloudy' or 'sunny'
    mean_temp = {'cloudy': 55.0, 'sunny': 75.0}[cloudy]     # mean tempereture when clould/ sunny
    scale_temp = {'cloudy': 10.0, 'sunny': 15.0}[cloudy]    # standar diviation for each conditions.
    temp = torch.distributions.Normal(mean_temp, scale_temp).rsample()
    return cloudy, temp.item()
```

To turn weather into a Pyro program, we’ll replace the `torch.distributions` with `pyro.distributions` and the `.sample()` and `.rsample()` calls with calls to `pyro.sample`, one of the core language primitives in Pyro. 

```python 
def weather_pyro():
  # Get a sample from a Bernoulli Distribution using Pyro.Sample 
  cloudy = pyro.sample('cloudy',pyro.distributions.Bernoulli(probs=0.3))
  cloudy = 'cloudy' if cloudy.item() == 1.0 else 'sunny'  # Transform cloudt into 'cloudy' or 'sunny'
  mean_temp = {'cloudy': 55.0, 'sunny': 75.0}[cloudy]     # mean tempereture when clould/ sunny
  scale_temp = {'cloudy': 10.0, 'sunny': 15.0}[cloudy]    # standar diviation for each conditions.

  temp = pyro.sample('temp', pyro.distributions.Normal(mean_temp,scale_temp))
  return cloudy, temp.item()
```


Procedurally, weather() is still a non-deterministic Python callable that returns two random samples. Because the randomness is now invoked with pyro.sample, however, it is much more than that. In particular `weather()` specifies a joint probability distribution over two named random variables: cloudy and temp. As such, it defines a probabilistic model that we can reason about using the techniques of probability theory. For example we might ask: if I observe a temperature of 70 degrees, how likely is it to be cloudy? How to formulate and answer these kinds of questions will be the subject of the next tutorial.

```python
def ice_cream_sales():
    cloudy, temp = weather()
    expected_sales = 200. if cloudy == 'sunny' and temp > 80.0 else 50.
    ice_cream = pyro.sample('ice_cream', pyro.distributions.Normal(expected_sales, 10.0))
    return ice_cream
``` 


In [11]:
# Define a generative model for weather - a stochastic functions. 
def weather():
    cloudy = torch.distributions.Bernoulli(0.3).sample()    # draw a sample from a Bernoulli 0 or 1 
    cloudy = 'cloudy' if cloudy.item() == 1.0 else 'sunny'  # Transform cloud into 'cloudy' or 'sunny'
    
    # based on the value of cloudy, choose a mean temperature, and a mean std. 
    mean_temp = {'cloudy': 55.0, 'sunny': 75.0}[cloudy]     # mean tempereture when clouldy/ sunny; Not
    scale_temp = {'cloudy': 10.0, 'sunny': 15.0}[cloudy]    # standar diviation for each conditions.

    # Draw a sample temperature from a Normal destribution with the selected mean. 
    temp = torch.distributions.Normal(mean_temp, scale_temp).rsample()
    
    return cloudy, temp.item()

In [21]:

cloudy, temperature = weather()
print(f"Weather is '{cloudy}' and the temerature is {temperature}")

Weather is 'sunny' and the temerature is 87.60794067382812


In [22]:
# Define the same model, but using pyro wraper function. 
# Important: Now pyro defines a joint probability distribution over two random variables
# 'cloudy' and 'temp' 

def weather_pyro():
  # Get a sample from a Bernoulli Distribution using Pyro.Sample 
  # A crucial difference is that this sample is named;  
  cloudy = pyro.sample('cloudy',pyro.distributions.Bernoulli(probs=0.3))
 
  cloudy = 'cloudy' if cloudy.item() == 1.0 else 'sunny'  # Transform cloudt into 'cloudy' or 'sunny'
  mean_temp = {'cloudy': 55.0, 'sunny': 75.0}[cloudy]     # mean tempereture when clould/ sunny
  scale_temp = {'cloudy': 10.0, 'sunny': 15.0}[cloudy]    # standar diviation for each conditions.

  # Here we use pyro.distributions.Normal instead, a wrapper to pytorch.distributions.Normal 
  # Again notice that the sample is named 
  temp = pyro.sample('temp', pyro.distributions.Normal(mean_temp,scale_temp))
  return cloudy, temp.item()

In [23]:
#
# Generate sample. 
for _ in range(3):
    print(weather_pyro())



('sunny', 76.62906646728516)
('cloudy', 42.30691146850586)
('sunny', 55.182289123535156)


We can use a stochastic function (like the `weather_pyro` function to create a composite models. For example, we can model the joint probability of cloudy, temperature and ice_cream sales). 

In [36]:
def ice_cream_sales(): 
  # Get a sample from weather_pyro model (i.e. the joint distribution of cloudy, temperature)
  cloudy, temperature = weather_pyro() 
  
  # defined expected sales (mean sales) as a function of cloudy and temperature variables. 
  expected_sales = 50.0 
  if (cloudy == 'sunny') and (temperature > 80.):
    expected_sales = 200.0

  # generate a sample for ice-cream sales; from a normab distribution with mean expected_sales and 10 variance.  
  ice_cream = pyro.sample('ice_cream', pyro.distributions.Normal(expected_sales,10))

  return cloudy, temperature, ice_cream.item() 


In [37]:
ice_cream_sales()

('sunny', 82.8541259765625, 204.81045532226562)

We can use `recursion` and `random control flow` (i.e. if statement where the condition is a random variable itself), to build arbitraryly complex models. When using recursion, we need to make sure we pass the variable index (i.e. t in the example below) to uniquely identify the variables being samples.  The `geometric` method below generates a sample indicating the number of trials needed to get a single success is a bernulli trial with success probability p.  

In [52]:
# p indicates the probability of success, t denotes the trial.
# The expeciment is number of trials until first success. 

def geometric(p, t=None):
  t = 0 if t==None else t 

  # Draw a sample from a bernulli distrupution with success probability p 
  # Notice that each variable is named; with an increasing index. 

  x = pyro.sample(f"x_{t}", pyro.distributions.Bernoulli(p))

  if (x.item()== 1):
    # return t, indicating that the t-th trial was successfull. 
    return t
  else :
    # current trial was not successful, recursivly call 
    return geometric(p,t+1)

In [56]:
geometric(0.3)

0

In [61]:
# Notice the pyro.sample generates a tensor, using .item() gets us the raw value of the tensor  
t = 100
x = pyro.sample(f"x_{t}", pyro.distributions.Bernoulli(0.5))

We can also define `Higher Order Stochastic Functions`; that would be stochastic functions that return stochastic functions.  

In [70]:
# This function return a sample from the random variable which is the product of two normaly distributed random variables.

def normal_product(loc, scale):
  # Generate one sample from a normal with mean loc, and variabce scale. 
  z1 = pyro.sample("z1", pyro.distributions.Normal(loc,scale))
  z2 = pyro.sample("z2", pyro.distributions.Normal(loc,scale))

  # Deterministically calculate the new variable z, as the product of two random values. 
  z = z1*z2 
  return z 

# Notice this is a Higher order function, it returns a lambda function.
def make_normal_normal():
    # Randomly 
    mu_latent = pyro.sample('mu_latent', pyro.distributions.Normal(0,1))

    # This lambda function takes as input a scale, and returns the output of teh normal_product.
    # notice fn is a reference to the lambda function. 
    fn = lambda scale : normal_product(mu_latent,scale)

    return fn 


In [71]:
# Notice that make_normal_normal() returns a function; which we invoke once it is returned. 
# When invoked, the stochastic function create 3 named random variables 'mu_latent', 'z1', 'z2'

make_normal_normal()(0.1)

tensor(0.1159)

#### 1.2 Universality 

In [45]:
t=1 
t=0 if t==None else t 

In [46]:
t

1

## An Introduction to Inference in Pyro